# Harmonization across studies with *NHLBI BioData Catalyst® (BDC) Powered by PIC-SURE*

This tutorial notebook will demonstrate how to query and work with the BDC studies, particularly cross-study harmonization. For a more step-by-step introduction to the python PIC-SURE API, see the `1_PICSURE_API_101.ipynb` notebook.

**Before running this notebook, please be sure to review the \"Get your security token\" documentation, which exists in the [`README.md` file](../README.md). It explains about how to get a security token, which is mandatory to access the databases.**

 -------   

# Environment set-up

### System requirements
- Python 3.6 or later
- pip python package manager, already available in most systems with a python interpreter installed


### Install packages

**Note that if you are using the dedicated PIC-SURE environment within the *BDC Powered by Seven Bridges* platform, the necessary packages have already been installed.**

In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# BDC Powered by Terra users uncomment the following line to specify package install location
# sys.path.insert(0, r"/home/jupyter/.local/lib/python3.7/site-packages")
!pip install matplotlib-venn
from matplotlib_venn import venn2


In [ ]:
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-biodatacatalyst-python-adapter-hpds.git

In [ ]:
import PicSureClient
import PicSureBdcAdapter

## Connecting to a PIC-SURE network

In [ ]:
PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
token_file = "token.txt"

with open(token_file, "r") as f:
    my_token = f.read()
    
bdc = PicSureBdcAdapter.Adapter(PICSURE_network_URL, my_token)

 -------   

## Harmonizing variables with PIC-SURE
One of the key challenges to conducting analyses with several studies is ensuring correct data harmonization, or combining of data from different sources. There are many harmonization techniques, but this notebook will demonstrate an approach to finding and extracting similar variables from different studies in PIC-SURE. Two examples of this will be shown:
1. Retrieving variables for *sex and gender* across studies with BMI
2. Harmonizing the *"orthopnea"* and *"pneumonia"* variables across studies


*For more information about the TOPMed DCC Harmonized Data Set in PIC-SURE, please refer to the `2_TOPMed_DCC_Harmonized_Variables_analysis.ipynb` notebook*

-----

## Sex and gender variables across studies
<font color='darkgreen'>**Goal: Create harmonized variables for sex and BMI which combine data from multiple studies**</font> 

These variables are labelled differently for each of these studies. For example, some use the keyword `sex` while others use `gender`. To acccount for these differences, we need to develop a way to search for multiple keywords at once.

Let's start by searching for `sex` and `gender` to gain a better understanding of the variables that exist in PIC-SURE with these terms. The dictionary().find method can take in regular expressions. Here, we take advantage of this by searching for both sex and gender in the same regular expression.


In [ ]:
sex_dictionary = bdc.useDictionary().dictionary().find('sex|gender')
sex_dataframe = sex_dictionary.dataframe()
print(sex_dataframe.shape)
sex_dataframe.head()

After reviewing the variables using the dataframe (or the [user interface](https://picsure.biodatacatalyst.nhlbi.nih.gov/psamaui/login)), let's say we are interested in sex/gender variables from the following studies:
- ECLIPSE (Evaluation of COPD Longitudinally to Identify Predictive Surrogate Endpoints) (phs001252)
- EOCOPD (Early Onset of COPD) (phs000946)

We can find the study IDs in the Data Access Dashboard in the user interface.

We have already used the PIC-SURE API dictionary object and find method to find variables which contain the keywords sex and gender. Let's filter these results to our desired studies of interest.

### Which sex variable in ECLIPSE should we use? 

First, we will filter our sex/gender variables to only look at those from the ECLIPSE study.

We can see that there are many sex/gender variables in ECLIPSE originating from various data tables.

We will examine the data associated with these variables to determine which one to use for our analysis.

In [ ]:
# which sex/gender variables are part of ECLIPSE?
eclipse_vars = sex_dataframe['columnmeta_study_id'].str.contains('phs001252')

# create a subset of only ECLIPSE sex/gender vars
eclipse_sex_df = sex_dataframe[eclipse_vars][['columnmeta_study_id', 'columnmeta_var_group_description', 'columnmeta_name', 'columnmeta_description', 'values', 'columnmeta_HPDS_PATH']]
eclipse_sex_df

We can see that the results there is a variable called `BICAT` or "Biomarker". This variable was selected because one of the values associated with this variable is "Sex hormone binding globulin" and was returned with our search. However, this is not a sex/gender variable and we should remove this.

In [ ]:
eclipse_sex_df = eclipse_sex_df[eclipse_sex_df['columnmeta_name'] != "BICAT"]

Many studies may have multiple variables with similar names and descriptions that come from different data tables. This will vary from study to study based on how the original study was conducted and organized. 

We will approach this complication by examining the data associated with each sex / gender variables in ECLIPSE and determining which would be the best fit for our analysis.

Because we want to examine the patient level data, we will create a query using all sex/gender variables in ECLIPSE. We will add these variables to our query using the `anyof` method, as we are interested in all observations with a value for any of our chosen variables.

In [ ]:
# Initialize a query
authPicSure = bdc.useAuthPicSure()
eclipse_sex_query = authPicSure.query()
eclipse_sex_query.anyof().add(eclipse_sex_df['columnmeta_HPDS_PATH'])
eclipse_sex_results = eclipse_sex_query.getResultsDataFrame()
eclipse_sex_results.head()

We used the `.getResultsDataFrame()` method to organize our participant level results in a dataframe that is easy to work with using pandas. By previewing the resulting dataframe, we can see that not all of the sex and gender variables are complete for our participant subset. 

Let's see which sex/gender variables are the most complete for our dataset by counting the number of NA values in each one. 

In [ ]:
# first let's see which sex/gender variables are the most complete for our dataset by counting the number of NA values
print(eclipse_sex_results.isna().sum())

We can see that there are a few variables with 0 NA values, we would like to focus on these.

In [ ]:
# let's focus on those variables which do not have any NA values
filtered_eclipse_sex_results = eclipse_sex_results.loc[:, eclipse_sex_results.isna().sum() == 0]
filtered_eclipse_sex_results

Now we have filtered our dataset to only include sex/gender variables in ECLIPSE which have complete data. Let's check to see if the values in each variable (column) differ, or if they all have the same data.

In [ ]:
# in this example, we notice that all our filtered variables have equivalent data for each patient:
for col in filtered_eclipse_sex_results.iloc[:, 4:]:
    print(col)
    print(filtered_eclipse_sex_results[col].unique())

We can see that all the values in the filtered, complete ECLIPSE sex/gender variables are the same, so we will simply select the first variable to use in our analysis.

We can use the `.varInfo()` method to learn more about a variable.

In [ ]:
eclipse_sex_var = filtered_eclipse_sex_results.columns[3]
sex_dictionary.varInfo(eclipse_sex_var)

### Which sex variable in EOCOPD should we use? 

We will take a similar approach as above, where we first filter our dataframe of sex/gender variables to only those from the EOCOPD study.

In [ ]:
# which sex/gender variables are part of EOCOPD?
eocopd_vars = sex_dataframe['studyId'].str.contains('phs000946')
sex_dataframe[eocopd_vars][['columnmeta_name', 'columnmeta_description']]

When reviewing the results from the search, we see that, unlike the ECLIPSE data, there is only one sex variable in EOCOPD. It is called 'GENDER'. Let's select this variable for filtering.

In [ ]:
# Save the GENDER var from EOCOPD
eocopd_sex_df = sex_dataframe[eocopd_vars][['columnmeta_study_id', 'columnmeta_var_group_description', 'columnmeta_name', 'columnmeta_description', 'values', 'columnmeta_HPDS_PATH']]
eocopd_sex_df = eocopd_sex_df[eocopd_sex_df["columnmeta_name"] == "GENDER"]
eocopd_sex_df

We will save the HPDS_PATH associated to this variable so we can add it to our query later.

In [ ]:
# save the HPDS path associated to our variable of interest
eocopd_sex_var = eocopd_sex_df[['columnmeta_HPDS_PATH']].iloc[0,0]
eocopd_sex_var

### Which BMI variables should we use?

We will follow the approach outlined above to first search our data dictionary for all variables containing the terms 'body mass index'. 

In [ ]:
bmi_dictionary = bdc.useDictionary().dictionary().find('body mass index')
bmi_dataframe = bmi_dictionary.dataframe()

We then filter to only those BMI variables which are part of ECLIPSE or EOCOPD and view the variables with their associated metadata. 

In [ ]:
# which bmi variables are part of ECLIPSE or EOCOPD?
bmi_vars = bmi_dataframe['studyId'].str.contains('phs001252|phs000946')

# create a subset of only the ECLIPSE / EOCOPD BMI vars
bmi_dataframe = bmi_dataframe[bmi_vars][['columnmeta_study_id', 'columnmeta_var_group_description', 'columnmeta_name', 'columnmeta_description', 'values', 'columnmeta_HPDS_PATH']]
bmi_dataframe.head(10)

By examining metadata like the dataTableName, description, and name, we are able to determine which variables we are interested in:
- Body Mass Index [BMI] for EOCOPD (\phs000946\pht005719\phv00266123\BMI\)
- Body mass index (kg/m2) from the bodycomp table for ECLIPSE (\phs001252\pht006373\phv00293610\VSBMI\) 

We will save the HPDS_PATHs associated to the BMI variables of interest so we can add it to our query later.

In [ ]:
# save HPDS_PATHs of interest
eocopd_bmi_var = "\phs000946\pht005719\phv00266123\BMI\\"
eclipse_bmi_var = "\phs001252\pht006373\phv00293610\VSBMI\\"

### Query PIC-SURE for selected variables

First, we create a new query:

In [ ]:
combined_query = authPicSure.query()

Next, we add the HPDS paths (concept paths) for our selected variabels of interest to the query.

As a reminder, these are the paths associated to the sex/gender and BMI variables from ECLIPSE and EOCOPD.

We use the `.anyof` method to add these variables to the query because we are interested in results with observations in any of the variables of interest. If you are interested in learning more about the other query methods available, see the `1_PICSURE_API_101.ipynb` notebook. 

In [ ]:
# Build query using selected concept paths
combined_query.anyof().add([eclipse_sex_var, eclipse_bmi_var, eocopd_sex_var, eocopd_bmi_var])

We will use the `.getResultsDataFrame()` method to get our results as a dataframe object for easy manipulation using pandas.

In [ ]:
# Get query results
combined_results = combined_query.getResultsDataFrame()
combined_results.head()

### Clean and wrangle query result dataframe

You may notice that the column names are not the most informative, we will rename them below.

In [ ]:
# rename columns
combined_results.columns = ['patient_id', 'parent_accession', 'topmed_accession', 'consent', 'eocopd_sex', 'eocopd_bmi', 'eclipse_bmi', 'eclipse_sex']
combined_results.head()

We will also create a 'study' column to easier identify from which study a participant's data originated from.

In [ ]:
# convert consent code to study name
combined_results['study'] = np.where(combined_results['consent'].str.contains('phs001252'), 'eclipse', 'eocopd')
combined_results.head()

Finally, we will harmonize the sex and bmi variables by combining them into one single 'sex' column.

Sex and BMI are reported in the same way in both EOCOPD and ECLIPSE, so we do not have to transform any of the participant level data.

In [ ]:
# combine sex variables
combined_results['sex'] = combined_results['eocopd_sex'].fillna(combined_results['eclipse_sex'])
combined_results.head()

In [ ]:
# combine bmi variables
combined_results['bmi'] = combined_results['eocopd_bmi'].fillna(combined_results['eclipse_bmi'])
combined_results.head()

### Analysis and visualization

In this example, we are interested in comparing mean BMI across different studies. We will calculate the mean for each group (ECLIPSE males, ECLIPSE females, EOCOPD males, EOCOPD females) and display the results as a table and a bar chart.

In [ ]:
# remove unneccessary columns and summarize
plotdf = combined_results[[ 'sex', 'study', 'bmi']]
plotdf = plotdf.groupby(['sex', 'study']).mean().unstack()
plotdf

In [ ]:
plotdf.plot(kind = 'bar')

-----

## Orthopnea variables across studies
<font color='darkgreen'>**Goal: Create harmonized variables for orthopnea and pneumonia which combine data from multiple studies, and compare participant distributions across these phenotypes**</font> 

In this example, we will be harmonizing variables related to `orthopnea`. Orthopnea is shortness of breath that occurs when individuals lie flat. Because of this, people with orthopnea have to sleep propped up in bed or sitting in a chair. You can read more about this condition [here](https://www.sleepfoundation.org/sleep-apnea/orthopnea).

Let's start by doing a search in PIC-SURE for concept paths containing `orthopnea`.

### Search and find orthopnea variables

In [ ]:
orthopnea_dictionary = bdc.useDictionary().dictionary().find("orthopnea")
orthopnea_dataframe = orthopnea_dictionary.dataframe()
print(orthopnea_dataframe.shape)
orthopnea_dataframe[['columnmeta_name', 'columnmeta_description', 'values', 'columnmeta_study_id']].head()

As shown in the resulting dataframe, orthopnea is often recorded as a something like `sleep on two or more pillows to help you breathe?`

Let's do a search for `pillows` in PIC-SURE to explore those concept paths.

In [ ]:
pillows_dictionary = bdc.useDictionary().dictionary().find("pillows")
pillows_dataframe = pillows_dictionary.dataframe()
pillows_dataframe[['columnmeta_name', 'columnmeta_description', 'values', 'columnmeta_study_id']].head()

As shown in the resulting dataframe, there are some concept paths related to orthopnea that were not captured by the previous `orthopnea` search, such as `Do you sleep on 2 or more pillows to improve your breathing?` from the Jackson Heart Study (JHS) Cohort (phs000286). In fact, the JHS dataset was not returned at all when we searched for `orthopnea`. 

This is problematic for researchers studying these types of variables, since the concept paths of interest may not always be captured by a single search term. To account for this, researchers must conduct exploratory searches to determine potential search terms.

Let's say we want to harmonize orthopnea variables from the following datasets: 
- FHS (phs000007)
- MESA (phs000209)
- WHI (phs000200)

We can first get all concept paths related to our terms of interest (`orthopnea` or `pillows`) and then filter to our studies of interest.

You can get the phs number associated with each study from the Data Access Dashboard.

In [ ]:
# search pic-sure for 'orthopnea' and 'pillows'
harmonized_dictionary = bdc.useDictionary().dictionary().find("orthopnea|pillows")
harmonized_df = harmonized_dictionary.dataframe()

# filter to variables found in FHS, MESA, WHI
harmonized_df = harmonized_df[harmonized_df['studyId'].str.contains('phs000007|phs000209|phs000200')]
print(harmonized_df.shape)
harmonized_df[['columnmeta_name', 'columnmeta_description', 'values', 'columnmeta_study_id']]

After browsing the available variables, we decided to choose the following:

In [ ]:
orthopnea_variables_of_interest = harmonized_df[harmonized_df['columnmeta_description'].isin(['ORTHOPNEA, RECENT', 
                                                                                   'F136 Orthopnea', 
                                                                                   'SLEEP ON 2+ PILLOWS TO HELP BREATHING'])]

# save variable paths for querying data
orthopnea_variable_paths_of_interest = orthopnea_variables_of_interest['columnmeta_HPDS_PATH'].tolist()

orthopnea_variables_of_interest[['columnmeta_name', 'columnmeta_description', 'values', 'columnmeta_study_id']]

### Search and find pneumonia variables

As part of our research, let's say we are interested in exploring the relationship between pneumonia and orthopnea. Let's save concept paths related to `pneumonia` as well. 

In [ ]:
# search pic-sure for 'pneumonia'
harmonized_dictionary = bdc.useDictionary().dictionary().find("pneumonia")
harmonized_df = harmonized_dictionary.dataframe()

# filter to variables found in FHS, MESA, WHI
harmonized_df = harmonized_df[harmonized_df['columnmeta_study_id'].str.contains('phs000007|phs000209|phs000200')]
print(harmonized_df.shape)
harmonized_df[['columnmeta_name', 'columnmeta_description', 'values', 'columnmeta_study_id', 'varId']]

After browsing the available variables, we decided to choose the following:

In [ ]:
pneumonia_variables_of_interest = harmonized_df[harmonized_df['varId'].isin(['phv00255144', 
                                                                             'phv00083774', 
                                                                             'phv00283208'])]

# save variable paths for querying data
pneumonia_variable_paths_of_interest = pneumonia_variables_of_interest['columnmeta_HPDS_PATH'].tolist()

pneumonia_variables_of_interest[['values', 'columnmeta_study_id', 'columnmeta_description', 'columnmeta_name', 'columnmeta_var_id']]

### Query PIC-SURE for selected variables

In [ ]:
# Initialize a query
authPicSure = bdc.useAuthPicSure()
orthopnea_query = authPicSure.query()
orthopnea_query.anyof().add(orthopnea_variable_paths_of_interest + pneumonia_variable_paths_of_interest)
orthopnea_results = orthopnea_query.getResultsDataFrame(low_memory=False)
orthopnea_results.head()

In [ ]:
# rename columns
orthopnea_results.columns = ['patient_id', 'parent_accession', 'topmed_accession', 'consent', 
                             'fhs_pneumonia', 'fhs_orthopnea', 'whi_pneumonia', 
                             'whi_orthopnea', 'mesa_pneumonia', 'mesa_orthopnea']
orthopnea_results.head()

### Clean and wrangle data

In [ ]:
# convert consent code to study name
orthopnea_results['study'] = np.where(orthopnea_results['consent'].str.contains('phs000007'), 'fhs', 
                                      np.where(orthopnea_results['consent'].str.contains('phs000209'), 'mesa', 'whi'))
orthopnea_results.head()

In [ ]:
# combine orthopnea variables
orthopnea_results['orthopnea'] = orthopnea_results['fhs_orthopnea'].fillna(orthopnea_results['whi_orthopnea'])
orthopnea_results['orthopnea'] = orthopnea_results['orthopnea'].fillna(orthopnea_results['mesa_orthopnea'])
orthopnea_results.head()



In [ ]:
# combine pneumonia variables
orthopnea_results['pneumonia'] = orthopnea_results['fhs_pneumonia'].fillna(orthopnea_results['whi_pneumonia'])
orthopnea_results['pneumonia'] = orthopnea_results['pneumonia'].fillna(orthopnea_results['mesa_pneumonia'])
orthopnea_results.head()

As you may have noticed, the orthopnea and pneumonia variables are encoded differently between these studies. 

This means that we see a range of vales that need to be harmonized (printed below)

We need to create a consistent encoding across these studies. 

In [ ]:
print(orthopnea_results.orthopnea.value_counts())
print('\n')
print(orthopnea_results.pneumonia.value_counts())

In [ ]:
len(np.unique(np.array(list(orthopnea_results['orthopnea']) + list(orthopnea_results['pneumonia']))))

We create a consistent mapping:

In [ ]:
mapping_df = pd.DataFrame({'raw': np.unique(np.array(list(orthopnea_results['orthopnea']) + list(orthopnea_results['pneumonia']))),
                           'harmonized': ['Unknown', 'Unknown', 'Unknown', 'Unknown', 'No', 'No', 'No', 'Yes', 'Yes', 'Unknown']})

value_mapping_dictionary = dict(zip(mapping_df.raw, mapping_df.harmonized))

mapping_df

And apply it to our hamonized orthopnea and pneumonia columns

In [ ]:
orthopnea_results.orthopnea = orthopnea_results.orthopnea.map(value_mapping_dictionary)
orthopnea_results.pneumonia = orthopnea_results.pneumonia.map(value_mapping_dictionary)

print(orthopnea_results.orthopnea.value_counts())
print('\n')
print(orthopnea_results.pneumonia.value_counts())

Now that the variables have been encoded in the same way, we can use this dataframe for analysis and visualizations. 

### Visualize

In [ ]:
final_df = orthopnea_results
final_df['orthopnea'] = final_df['orthopnea']. fillna('Unknown')
final_df['pneumonia'] = final_df['pneumonia']. fillna('Unknown')

In [ ]:
# Filters to apply to the final_df to create figures
pneu_yes = final_df['pneumonia'] == 'Yes'
pneu_no = final_df['pneumonia'] == 'No'
ortho_yes = final_df['orthopnea'] == 'Yes'
ortho_no = final_df['orthopnea'] == 'No'
fhs = final_df['study'] == 'fhs'
mesa = final_df['study'] == 'mesa'
whi = final_df['study'] == 'whi'

In [ ]:
# FHS figure
pneu_count =  final_df[pneu_yes & ortho_no & fhs].count()['patient_id']
ortho_count = final_df[pneu_no & ortho_yes & fhs].count()['patient_id']
both_count =  final_df[pneu_yes & ortho_yes & fhs].count()['patient_id']
venn2(subsets = (pneu_count, ortho_count, both_count), set_labels = ('Pneumonia', 'Orthopnea'))
plt.title("FHS Dataset")
plt.show()

# MESA figure
pneu_count =  final_df[pneu_yes & ortho_no & mesa].count()['patient_id']
ortho_count = final_df[pneu_no & ortho_yes & mesa].count()['patient_id']
both_count =  final_df[pneu_yes & ortho_yes & mesa].count()['patient_id']
venn2(subsets = (pneu_count, ortho_count, both_count), set_labels = ('Pneumonia', 'Orthopnea'))
plt.title("MESA Dataset")
plt.show()

# WHI figure
pneu_count =  final_df[pneu_yes & ortho_no & whi].count()['patient_id']
ortho_count = final_df[pneu_no & ortho_yes & whi].count()['patient_id']
both_count =  final_df[pneu_yes & ortho_yes & whi].count()['patient_id']
venn2(subsets = (pneu_count, ortho_count, both_count), set_labels = ('Pneumonia', 'Orthopnea'))
plt.title("WHI Dataset")
plt.show()

# Combined figure
pneu_count =  final_df[pneu_yes & ortho_no].count()['patient_id']
ortho_count = final_df[pneu_no & ortho_yes].count()['patient_id']
both_count =  final_df[pneu_yes & ortho_yes].count()['patient_id']
venn2(subsets = (pneu_count, ortho_count, both_count), set_labels = ('Pneumonia', 'Orthopnea'))
plt.title("Combined results across FHS, MESA, and WHI")
plt.show()